<a href="https://colab.research.google.com/github/AbeHandler/AbeHandler.github.io/blob/master/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Intro

In this notebook you will get some practice with a span tagging task (also called token classification) using the popular [Huggingface](https://huggingface.co/) library which makes it (fairly) easy to use large transformer models for NLP tasks.

The data that we will be using for this assignment comes from [RedHOT](https://arxiv.org/abs/2210.06331), a corpus of medical posts from reddit labeled with "PIO" tags where P stands for "population," I stands for "intervention" and O stands for "outcome". PIO frames are often used to create structured representations of text from medical studies.

This dataset is motivated by a desire to catch medical disinformation online. For example, you could use PIO elements to try to find posts which propose (falsely) that [bleach](https://www.theguardian.com/world/2020/sep/19/bleach-miracle-cure-amazon-covid) is a cure for COVID (which is false).

Credits: this problem set draws from tutorials from [Huggingface](https://huggingface.co/docs/transformers/tasks/token_classification) and [wandb](https://wandb.ai/mostafaibrahim17/ml-articles/reports/Named-Entity-Recognition-With-HuggingFace-Using-PyTorch-and-W-B--Vmlldzo0NDgzODA2).

#### Q1

Data science does not happen in a vacuum. It's usually a good idea to really understand your data before you even get started with modeling. When you are working in a supervised setting, you should spend some time understanding what your labels mean!

Get started by gaining a little bit of familiarity with the PIO tagging scheme by looking at the example in [Table 1](https://arxiv.org/pdf/2210.06331.pdf) of the RedHOT paper. Then define each of the following in your own words by filling out the cell below. You can write one clear sentence for each of these.

**population** _your answer here_

**intervention** _your answer here_

**outcome** _your answer here_

#### Q2

Find a post on r/AskDocs which references a population, intervention or outcome (or all of the above). Include the text of the post below, and describe the population interention and outcome in the post. If the post is really long, pick the most important one or two sentences.

_Your answer here_

### Coding preliminaries

In order complete the assignment you will need to install the necessary packages and set up your runtime. There are three steps here:

1. A the menu at the top, go to "Runtime" and pick "Change runtime type" and pick "T4." T4 is a kind of GPU made by Nvidia. When you select this runtime your collab notebook will connect to a T4 GPU. Google gives you a small number of GPU credits as part of a free colab account. It is not really enough for production or research use, but it is an easy way to get started with GPU programming for a problem set. If you skip this step your model will train on a CPU which will take a really long time.

2.  Uncomment the next line and install the required dependencies using pip. You don't have to use a GPU runtime to complete this assignment.

3. Upload your `train.conll.txt` and `validation.conll.txt` files to colab. You can find instructions on how to do that [here](https://docs.google.com/document/d/1f1DQI_QUQId4x8fFQQ59cdenAZWWq9eK6HMyYchdpQM/edit?usp=sharing).

In [2]:
# ! pip install transformers datasets evaluate seqeval transformers[torch]

In [3]:
# run this cell to import the needed packages
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
from datasets import load_dataset, load_metric, Dataset, DatasetDict
import numpy as np
from seqeval.metrics import f1_score, precision_score, recall_score, classification_report


#### Q3: Understanding the CONLL format

In the lectures for this class, we have been talking about "supervised" learning for span tagging (aka token classification) kind of abstractly. But what does "supervised" data actually look like for span tagging tasks? Well, one common format is called CONLL. Each token in a CONLL file is listed on its own line, and its label is listed to the right (separated by a space delimiter).

In our setting, the tag labels are "P" for population, "I" for intervention and "O" for outcome. Also, the first token in a span gets the tag B-P (for beggining) and all other tags in the labeled span get the tag I-P. So for example the string "Im also HLA-B27 positive" gets tagged as follows.

```
I O
m O
also O
HLA B-P
- I-P
B27 I-P
positive O
```


The next cell reads in a conll file. You can leave it unchanged.

In [4]:
def read_conll_file(file_path):
    lines = 0
    with open(file_path, "r") as f:
        content = f.read().strip()
        sentences = content.split("\n\n")
        data = []
        for sentence in sentences:
            tokens = sentence.split("\n")
            token_data = []
            for token in tokens:
                token_data.append(token.split())
            data.append(token_data)
            lines += 1
    return data


train_data = read_conll_file("train.conll.txt")
validation_data = read_conll_file("validation.conll.txt")

Question 3: _Explain what the code in the cell above is doing in your own words_

#### Converting to a Dataset object

Huggingface transformers models often use a dataset object from the Huggingface [Datasets](https://huggingface.co/docs/datasets/index) library. The code below simply converts the connl data into this format. There is not much you need to do for this. To some extent this is boilerplate preprocessing needed to get the RedHOT data data ready to use Huggingface library.

In [5]:
def convert_to_dataset(data, label_map):
    formatted_data = {"tokens": [], "ner_tags": []}
    for sentence in data:
        if sentence[0] != []:
            try:
                tokens = [token_data[0] for token_data in sentence]
                ner_tags = [label_map[token_data[1]] for token_data in sentence]
                formatted_data["tokens"].append(tokens)
                formatted_data["ner_tags"].append(ner_tags)
            except IndexError:
                pass
            except KeyError:
                print(sentence)
    return Dataset.from_dict(formatted_data)


label_set = set()
for sentence in train_data:
    for token_data in sentence:
        try:
            label_set.add(token_data[1])
        except IndexError:
            pass

label_list = sorted(list(label_set))
label_map = {label: i for i, label in enumerate(label_list)}

train_dataset = convert_to_dataset(train_data, label_map)
validation_dataset = convert_to_dataset(validation_data, label_map)
#test_dataset = convert_to_dataset(test_data, label_map)

datasets = DatasetDict({
    "train": train_dataset,
    "validation": validation_dataset,
    #"test": test_dataset,
})


### Inspect the data

In [25]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(label_list))


Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
def compute_metrics(eval_prediction):
    predictions, labels = eval_prediction
    predictions = np.argmax(predictions, axis=2)


    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    return {
        "precision": precision_score(true_labels, true_predictions),
        "recall": recall_score(true_labels, true_predictions),
        "f1": f1_score(true_labels, true_predictions),
        # "classification_report": classification_report(true_labels, true_predictions),
    }


def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True, padding=True
    )
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [27]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

training_args = TrainingArguments(
    output_dir="./results",
    eval_steps=500,
    save_steps=500,
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_steps=100,
    learning_rate=5e-5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)


Map:   0%|          | 0/4590 [00:00<?, ? examples/s]

Map:   0%|          | 0/530 [00:00<?, ? examples/s]

In [28]:
def data_collator(data):
    input_ids = [torch.tensor(item["input_ids"]) for item in data]
    attention_mask = [torch.tensor(item["attention_mask"]) for item in data]
    labels = [torch.tensor(item["labels"]) for item in data]


    input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask = torch.nn.utils.rnn.pad_sequence(attention_mask, batch_first=True, padding_value=0)
    labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=-100)


    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

#### Ok, train!

You are finally ready to train your model. In may ways, this is kind of the easy part. Once you have set all of the training configurations there is just one line of code that you need to call to actually train the model, shown below. This will take about 15 minutes to run in a T4 runtime. Go take a walk :). Waiting for GPUs is alas a big part of modern data science.

In [29]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.121400,0.092688,0.173077,0.102857,0.129032


KeyboardInterrupt: ignored

#### Check the results

Once you train a model, you are probably curious to see how it will do on real datasets. The next cell shows how to run the trained model. Let's test how it does on the example from the Table in the paper which we examined earlier in the problem set.

In [48]:
sentence = '''I’ve had costochondritis for a while, usually comes and goes.
Done all the heart/lung checks all clear. I’ve just recovered covid and what I’m
left with is chest pain/pressure. I mean it could
be a costo flare up which makes sense, but also
been reading about myocarditis after covid and
I’m worried, how can I tell which is which?'''


# tokenize the input
tokenized_input = tokenizer(sentence, return_tensors="pt").to(model.device)

# make predictions about the input
outputs = model(**tokenized_input)

# get the predicted labels for each token
predicted_labels = outputs.logits.argmax(-1)[0]

named_entities = [(tokenizer.decode([token]), label) for token, label in zip(tokenized_input["input_ids"][0], predicted_labels) if label != 0 and label != label_map['O']]

print("Named Entities - Example 1:", named_entities)


Named Entities - Example 1: [('cost', tensor(2, device='cuda:0')), ('##och', tensor(5, device='cuda:0')), ('##ond', tensor(5, device='cuda:0')), ('##rit', tensor(5, device='cuda:0')), ('##is', tensor(5, device='cuda:0')), ('chest', tensor(1, device='cuda:0')), ('pain', tensor(1, device='cuda:0')), ('pressure', tensor(1, device='cuda:0'))]


#### Error analysis

- What is one thing the model got right?
- What is one thing the model got wrong?

In [ ]:
#### Another error analysis

So far, you have just seen one example. Go to the [AskDocs](https://www.reddit.com/r/AskDocs/) subreddit and find another medical question.
Post the text of the question below and answer the questions again.

- What is one thing the model got right?
- What is one thing the model got wrong?

In [47]:
label_map

{'B-I': 0, 'B-O': 1, 'B-P': 2, 'I-I': 3, 'I-O': 4, 'I-P': 5, 'O': 6}

In [19]:
# dataset["train"]

Dataset({
    features: ['ner_tags', 'tokens', 'id'],
    num_rows: 2265
})

### BIO tags question

assign your own tags

### Should you keep training?

### Load the tokenizer


In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

Tokenizer question here

- [word piece](https://huggingface.co/learn/nlp-course/chapter6/6?fw=pt)
- CLS and SEP

Reading the documentation questions

In [6]:
# Example text

text = "He told me that the Nissan funduplication surgery was a success for 80% of cases after 15 years."

inputs = tokenizer(text, return_tensors="pt")
tokenizer.convert_ids_to_tokens(inputs["input_ids"].squeeze())

['[CLS]',
 'he',
 'told',
 'me',
 'that',
 'the',
 'nissan',
 'fund',
 '##up',
 '##lica',
 '##tion',
 'surgery',
 'was',
 'a',
 'success',
 'for',
 '80',
 '%',
 'of',
 'cases',
 'after',
 '15',
 'years',
 '.',
 '[SEP]']

In [9]:
V = tokenizer.vocab
"he" in V

True

In [12]:
len(V)

30522

In [13]:
"funduplication" in V # sparsity of text

False

In [13]:
example = wnut["train"][0]
example

{'id': '0',
 'tokens': ['@paulwalk',
  'It',
  "'s",
  'the',
  'view',
  'from',
  'where',
  'I',
  "'m",
  'living',
  'for',
  'two',
  'weeks',
  '.',
  'Empire',
  'State',
  'Building',
  '=',
  'ESB',
  '.',
  'Pretty',
  'bad',
  'storm',
  'here',
  'last',
  'evening',
  '.'],
 'ner_tags': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  8,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

In [6]:
example = wnut["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['[CLS]',
 '@',
 'paul',
 '##walk',
 'it',
 "'",
 's',
 'the',
 'view',
 'from',
 'where',
 'i',
 "'",
 'm',
 'living',
 'for',
 'two',
 'weeks',
 '.',
 'empire',
 'state',
 'building',
 '=',
 'es',
 '##b',
 '.',
 'pretty',
 'bad',
 'storm',
 'here',
 'last',
 'evening',
 '.',
 '[SEP]']

In [7]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_wnut = wnut.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/3394 [00:00<?, ? examples/s]

Map:   0%|          | 0/1009 [00:00<?, ? examples/s]

Map:   0%|          | 0/1287 [00:00<?, ? examples/s]

In [8]:
import evaluate

seqeval = evaluate.load("seqeval")


In [9]:
import numpy as np

labels = [label_list[i] for i in example[f"ner_tags"]]

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [10]:
id2label = {
    0: "O",
    1: "B-corporation",
    2: "I-corporation",
    3: "B-creative-work",
    4: "I-creative-work",
    5: "B-group",
    6: "I-group",
    7: "B-location",
    8: "I-location",
    9: "B-person",
    10: "I-person",
    11: "B-product",
    12: "I-product",
}
label2id = {
    "O": 0,
    "B-corporation": 1,
    "I-corporation": 2,
    "B-creative-work": 3,
    "I-creative-work": 4,
    "B-group": 5,
    "I-group": 6,
    "B-location": 7,
    "I-location": 8,
    "B-person": 9,
    "I-person": 10,
    "B-product": 11,
    "I-product": 12,
}

from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=13, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [19]:
training_args = TrainingArguments(
    output_dir="my_awesome_wnut_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_wnut["train"],
    eval_dataset=tokenized_wnut["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.267852,0.547718,0.367006,0.439512,0.945022
2,No log,0.280791,0.598160,0.361446,0.450607,0.946518
3,0.072800,0.320172,0.595016,0.354032,0.443928,0.945919
4,0.072800,0.314644,0.552910,0.387396,0.455586,0.947031
5,0.033100,0.304680,0.535055,0.403151,0.459831,0.946261


TrainOutput(global_step=1065, training_loss=0.05105608259568192, metrics={'train_runtime': 132.4988, 'train_samples_per_second': 128.077, 'train_steps_per_second': 8.038, 'total_flos': 230447974115460.0, 'train_loss': 0.05105608259568192, 'epoch': 5.0})

In [13]:
model.save_pretrained('./my_awesome_wnut_model')
tokenizer.save_pretrained('./my_awesome_wnut_model')


('./my_awesome_wnut_model/tokenizer_config.json',
 './my_awesome_wnut_model/special_tokens_map.json',
 './my_awesome_wnut_model/vocab.txt',
 './my_awesome_wnut_model/added_tokens.json',
 './my_awesome_wnut_model/tokenizer.json')

In [14]:
from transformers import pipeline

text = "The Golden State Warriors are an American professional basketball team based in San Francisco."

[{'entity': 'B-location',
  'score': 0.49131355,
  'index': 2,
  'word': 'golden',
  'start': 4,
  'end': 10},
 {'entity': 'B-location',
  'score': 0.234397,
  'index': 3,
  'word': 'state',
  'start': 11,
  'end': 16},
 {'entity': 'B-location',
  'score': 0.55225897,
  'index': 13,
  'word': 'san',
  'start': 80,
  'end': 83},
 {'entity': 'B-location',
  'score': 0.48205167,
  'index': 14,
  'word': 'francisco',
  'start': 84,
  'end': 93}]

In [24]:
import torch
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification


text = "I love New York"

tokenizer = AutoTokenizer.from_pretrained("./my_awesome_wnut_model")
inputs = tokenizer(text, return_tensors="pt")

model = AutoModelForTokenClassification.from_pretrained("./my_awesome_wnut_model")
with torch.no_grad():
    logits = model(**inputs).logits

In [25]:
predictions = torch.argmax(logits, dim=2)
predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]
predicted_token_class

['O', 'O', 'O', 'B-location', 'B-location', 'O']

In [29]:
inputs["input_ids"]

tensor([[ 101, 1045, 2293, 2047, 2259,  102]])

In [30]:
tokenizer("this is a test")

{'input_ids': [101, 2023, 2003, 1037, 3231, 102], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [32]:
# Get predictions
with torch.no_grad():
    logits = model(**inputs).logits

# Get the predicted token ids
predicted_token_ids = torch.argmax(logits, dim=-1)

tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"].squeeze())

# Map tokens to their predictions
for token, prediction in zip(tokens, predicted_token_ids.squeeze().tolist()):
    print(f"{token}: {prediction}")

[CLS]: 0
i: 0
love: 0
new: 7
york: 7
[SEP]: 0


In [33]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"].squeeze())

['[CLS]', 'i', 'love', 'new', 'york', '[SEP]']